### google 

## 1. rss (키워드 검색)

In [ ]:

import feedparser
import ssl
import urllib.request
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
corps = set()

keyword = '오늘날씨'

rss_url = f"https://news.google.com/rss/search?q={keyword}&hl=ko&gl=KR&ceid=KR:ko"
# rss_url = f"https://news.google.com/rss/search?q={keyword}"
feed = feedparser.parse(rss_url)

results = []
for entry in feed.entries:
    result = {
        'title': entry.title,
        'link': entry.link,
        'published': entry.published,
        'corp' : entry.title.split(' - ')[1]
    }
    results.append(result)
    corps.add(entry.title.split(' - ')[1])

print(len(results))

In [ ]:
import requests
from bs4 import BeautifulSoup, Comment
import feedparser
from newspaper import Article
import datetime
from fake_useragent import UserAgent


class NewsCrawler():
    def __init__(self):
        self.user_agent = UserAgent()
        self.headers = None

    def extract_tag(self, item, tag):
        [elem.extract() for elem in item(tag)]


    def extract_comments(self, item):
        for element in item(text=lambda text: isinstance(text, Comment)):
            element.extract()
            
    def get_headers(self, shuffle_header=False):
        if self.headers is None or shuffle_header:
            self.headers = {
                'User-Agent': self.user_agent.random,
            }
        return self.headers


    def crawl_url(self, url, shuffle_header=False, language='ko'):
        try:
            # article title
            article = Article(url, language=language)
            article.download()
            article.parse()
            if not article.url:
                print("[ERROR] no url found..")
                return None        

            data = {}
            data['title'] = article.title
            data['image'] = article.meta_img
            data['crawl_url'] = url
            data['url'] = article.url

            print(article.url)
            if article.publish_date:
                data['published_at'] = article.publish_date
            else:
                data['published_at'] = datetime.datetime.now()

            res = requests.get(url, headers=self.get_headers(shuffle_header=shuffle_header))
            soup = BeautifulSoup(res.text, 'html.parser')

            for script in soup(['script', 'style']):
                script.decompose()

            body = soup.find("div", itemprop="articleBody")

            if body is None:
                body = soup.select('#articleBodyContents')
                if len(body) > 0:
                    body = body[0]
                    self.extract_comments(body)
                    self.extract_tag(body, 'img')
                    self.extract_tag(body, 'div')
                    body = str(body)
                else:
                    body = article.text
                    body = body.replace('\n','<br>')
            else:
                self.extract_comments(body)
                self.extract_tag(body, 'img')
                self.extract_tag(body, 'div')
                body = str(body)

            data['text'] = article.text
            data['body'] = body
            data['language'] = language

        except Exception as e:
            print(e)
            data = None

        return data
    
    def crawl_rss(self, rss_url, shuffle_header=False, language='ko'):
        feed = feedparser.parse(rss_url)
        data_list = []
        for post in feed.entries:
            if post.link:
                data_list.append(self.crawl_url(post.link, shuffle_header, language))

        return data_list

In [ ]:
sample_url = """https://news.google.com/rss/articles/CBMiMGh0dHBzOi8vd3d3Lm5ld3NwaW0uY29tL25ld3Mvdmlldy8yMDI0MDUwODAwMDAxOdIBMWh0dHBzOi8vbS5uZXdzcGltLmNvbS9uZXdzYW1wL3ZpZXcvMjAyNDA1MDgwMDAwMTk?oc=5
"""

c = NewsCrawler()
# tmp = c.crawl_url(sample_url)


In [ ]:
import pandas as pd

In [ ]:
url_df = pd.DataFrame(columns=['title','image','crawl_url','url','published_at','text','body'])
url_df


In [ ]:

c = NewsCrawler()

In [ ]:
# for r in results:
#     tmp = c.crawl_url(r['link'])
#     length = len(url_df)+1
    
#     url_df.loc[length, 'title'] = tmp['title']
#     url_df.loc[length, 'image'] = tmp['image']
#     url_df.loc[length, 'crawl_url'] = tmp['crawl_url']
#     url_df.loc[length, 'url'] = tmp['url']
#     url_df.loc[length, 'published_at'] = tmp['published_at']
#     url_df.loc[length, 'text'] = tmp['text']
#     url_df.loc[length, 'body'] = tmp['body']
    
# url_df

In [ ]:
import re


In [ ]:
# custom_words = ['기자', '일보', '앵커']   


In [ ]:
# for idx, row in url_df.loc[:1].iterrows():
#     url = row['url']
#     print(row['text'])
#     text = row['text'].replace('\n', '')
#     pattern1 = r'\b(?:' + '|'.join(list(corps)) + r')\b'
#     preprocessed_text = re.sub(pattern1, '', text)
#     pattern2 = r'[^\w\s,~.\'"]'
#     preprocessed_text = re.sub(pattern2, '', preprocessed_text)
#     pattern3 = r'\b(?:\w+ 앵커|\w+ 기자|\w+일보)\b'
#     preprocessed_text = re.sub(pattern3, '', preprocessed_text)

#     print(preprocessed_text)

### keyword 추출

In [ ]:

from mecab import MeCab as mecab
mecab = mecab()

## mecab-co-dic 품사 태그

    - NNG : 일반명사
    - NNP : 고유명사
    - NNB : 의존명사
    - NNBC : 단위를 나타내는 명사
    - NR : 수사
    - NP : 대명사

In [ ]:
def get_pos_tagging(query):
    print(f" 원 쿼리 -> {query}")
    mecab_result = mecab.pos(query)
    print(f" mecab pos tagging -> {mecab_result}")
    result = []
    combined_word = ''

    cur = 0
    prev_pos = None
    while cur < len(mecab_result):
        word, pos = mecab_result[cur]
        if pos == 'SN':
            combined_word += word
            nxt = cur + 1
            while nxt < len(mecab_result) and mecab_result[nxt][1] == 'SN':
                combined_word += mecab_result[nxt][0]
                nxt += 1
                
            while nxt < len(mecab_result) and mecab_result[nxt][1] in ['NNG', 'NNP', 'NNB', 'NNBC', 'NR', 'NP']:
                if mecab_result[nxt-1][1] in ['NNG', 'NNP', 'NNB', 'NNBC', 'NR', 'NP']:
                    break
                combined_word += mecab_result[nxt][0]
                nxt += 1
            result.append(combined_word)
            combined_word = ''
            cur = nxt
        elif pos in ['MAG', 'NNG', 'NNP', 'NNB', 'NNBC', 'NR', 'NP']:
            combined_word = word
            result.append(combined_word)
            nxt = cur + 1
            combined_word = ''
            cur = nxt
        else:
            cur += 1

    print(f" 키워드 추출 -> {result}")
    return result

In [ ]:
result = get_pos_tagging('내일 서울 날씨 알려줘')

### 검색어 가져와서 rss 검색

In [ ]:
import random

In [ ]:
keyword_combination_1= '+'.join(random.sample(result, len(result)))
print(keyword_combination_1)

In [ ]:
keyword_combination_2= '+'.join(random.sample(result, len(result)))
print(keyword_combination_2)

In [ ]:
keyword_combination_3= '+'.join(random.sample(result, len(result)))
print(keyword_combination_3)

In [ ]:
def search_rss_keyword(keyword_combination):
    corps = set()
    rss_url = f"https://news.google.com/rss/search?q={keyword_combination}&hl=ko&gl=KR&ceid=KR:ko"
    # rss_url = f"https://news.google.com/rss/search?q={keyword}"
    feed = feedparser.parse(rss_url)

    results = []
    for entry in feed.entries:
        result = {
            'title': entry.title,
            'link': entry.link,
            'published': entry.published,
            'corp' : entry.title.split(' - ')[1]
        }
        results.append(result)
        corps.add(entry.title.split(' - ')[1])
        
    return results
    

In [ ]:
case1 = search_rss_keyword(keyword_combination_1)
case2 = search_rss_keyword(keyword_combination_2)
case3 = search_rss_keyword(keyword_combination_3)

In [ ]:
url_call_df_1= pd.DataFrame(case1)
url_call_df_1

In [ ]:
url_call_df_2= pd.DataFrame(case2)
url_call_df_2

In [ ]:
url_call_df_3= pd.DataFrame(case3)
url_call_df_3

In [ ]:
final_url_df = pd.merge(pd.merge(url_call_df_1, url_call_df_2), url_call_df_3)
final_url_df

### ML 기반 유사도 높은 문서 5개 가져오기

In [ ]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer('sentence-transformers/stsb-xlm-r-multilingual')

In [ ]:
train_data = final_url_df.copy()

In [ ]:
train_data['embedding'] = train_data.apply(lambda row: model.encode(row.title), axis = 1)

In [ ]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [ ]:
train_data

In [ ]:
query = "내일 서울 날씨 알려줘"

embedding = model.encode(query)
train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
sim_result_df = train_data.sort_values(by='score', ascending=False)[:5].reset_index(drop=True)
sim_result_df.loc[:5]

In [ ]:
content_df = pd.DataFrame()

In [ ]:
for idx, row in sim_result_df[:10].iterrows():
    tmp = c.crawl_url(row['link'])
    length = len(content_df)+1
    
    try :
        content_df.loc[length, 'title'] = tmp['title']
        content_df.loc[length, 'url'] = tmp['url']
        content_df.loc[length, 'published_at'] = tmp['published_at']
        content_df.loc[length, 'text'] = tmp['text'].replace('\n', '')
    except:
        pass
    
content_df

### LLM 요약

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
import json

In [ ]:
content_df

### LLM (Retrieval)

In [ ]:
from langchain_community.document_loaders import JSONLoader

In [ ]:
from pprint import pprint

In [ ]:
result_data_json= content_df.to_json(orient='records', force_ascii=False, date_format='iso')

In [ ]:
result_json = json.loads(result_data_json)
pprint(result_json)

### LLM (요약) 

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

In [ ]:
model = ChatOpenAI(model='gpt-3.5-turbo',
                   temperature=0)

In [ ]:
class SummaryObject(BaseModel):
    title: str = Field(description="본문의 title")
    url: str = Field(description="본문에 해당하는 url")
    summary: str = Field(description="본문을 요약한 텍스트")

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import ConversationChain

In [ ]:
parser = JsonOutputParser(pydantic_object=SummaryObject)

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert summarizer."
            "Summarizes text content related to a given query."
            ),
            (
            "user",
            "Please summarize the sentences according to the following requests."
            "\nRequest:\n"
            "Please summarize the data in 1 to 3 lines as requested below."
            "\nRequest:\n"
            "For weather-related inquiries, please summarize the published_at closest to {today}."
            "Please indicate the URL of the referenced content as the source."
            "Only content that can answer the provided query should be summarized in Korean."
            "Be sure to include the URL related to the referenced content as the source."
            "\n\nQuery: {query}\n\nContext: {context}\n\nSummary:",
                    ),
    ]
)
chain = prompt | llm
response= chain.invoke({"query": query, "context" : result_json, "today" : datetime.date.today()})

print(response.content)